In [1]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import os
import sys

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

In [2]:
# MPC import
from LinearMPC.MPCVelControl import MPCVelControl
from PIControl.PIControl import PIControl
import numpy as np
from src.rocket import Rocket
from src.vel_rocket_vis import RocketVis

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

In [5]:
Ts = 0.05
sim_time = 20
H = 5.0
x0 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 50, 50, 100])  # initial state
pos_target = np.array([0, 0, 10.0])

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
pos_controller = PIControl(pos_target)
mpc = MPCVelControl().new_controller(rocket, Ts, H)

t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, ref = rocket.simulate_control(
    mpc, sim_time, H, x0, pos_control=pos_controller, method="nonlinear"
)

vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1.0
vis.animate(
    t_cl[:-1],
    x_cl[:, :-1],
    u_cl,
    Ref=ref[:, :-1],
    T_ol=t_ol[..., :-1],
    X_ol=x_ol,
    U_ol=u_ol,
);

c:\Users\loeff\miniconda3\envs\mpc2025\Lib\site-packages\cvxpy\reductions\solvers\solving_chain_utils.py:30: UserWarning: The problem includes expressions that don't support CPP backend. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(







 State beta violation: -0.20 < -0.17, 
 State alpha violation: 0.20 > 0.17, 

 State beta violation: -0.31 < -0.17, 
 State alpha violation: 0.32 > 0.17, 

 State beta violation: -0.44 < -0.17, 
 State alpha violation: 0.47 > 0.17, 

 State beta violation: -0.56 < -0.17, 
 State alpha violation: 0.66 > 0.17, 

 State beta violation: -0.66 < -0.17, 
 State alpha violation: 0.85 > 0.17, 

 State beta violation: -0.73 < -0.17, 
 State alpha violation: 1.02 > 0.17, 

 State beta violation: -0.76 < -0.17, 
 State alpha violation: 1.17 > 0.17, 

 State beta violation: -0.79 < -0.17, 
 State alpha violation: 1.27 > 0.17, 

 State beta violation: -0.80 < -0.17, 
 State alpha violation: 1.33 > 0.17, 

 State beta violation: -0.80 < -0.17, 
 State alpha violation: 1.35 > 0.17, 

 State beta violation: -0.81 < -0.17, 
 State alpha violation: 1.32 > 0.17, 

 State beta violation: -0.82 < -0.17, 
 State alpha violation: 1.25 > 0.17, 

 State beta violation: -0.83 < -0.17, 
 State alpha violati

AppLayout(children=(HBox(children=(Play(value=0, description='Press play', max=399, step=2), IntSlider(value=0…

### OL xvel

In [7]:
from LinearMPC.MPCControl_xvel import MPCControl_xvel
import cvxpy as cp
Ts = 0.05
sim_time = 30
H = 7
x_target = np.array([0,0,5])
# Use a state closer to equilibrium (with small velocities)
x0 = np.array([0, 0, 0])   # initial state

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
xs, us = rocket.trim()

A, B = rocket.linearize(xs, us)

mpc_x = MPCControl_xvel(A, B, xs, us, Ts, H) # Full 12x12 A and 12x4 B matrices
u0, x_traj, u_traj = mpc_x.get_u(x0,x_target)

print("u0 = ",u0)

u0 =  [-0.26]
